In [1]:
# Install AWS SDK to communicate with S3
!pip install boto3

# Follow setup here https://github.com/boto/boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 32.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 3.4 MB/s eta 0:00:00


In [2]:
import boto3
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

pawel123r


In [4]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [7]:
s3 = boto3.client('s3')
BUCKET_NAME = 'pawel123r'
OBJECT_NAME = 'test.txt'
FILE_NAME = 'random.txt'

with open(FILE_NAME, "rb") as f:
    s3.upload_fileobj(f, BUCKET_NAME, OBJECT_NAME)

In [8]:
upload_file(FILE_NAME, BUCKET_NAME, object_name=OBJECT_NAME.split('.')[0] + '_v1.txt')

True

In [18]:
import pandas as pd

def txt2dataframe(txt_path):
    return pd.read_csv(txt_path, names=['input', 'label'], delimiter=';')

train = txt2dataframe('data/train.txt')
train.head()

,input,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [19]:
val = txt2dataframe('data/val.txt')
val.head()

,input,label
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [20]:
train.to_csv('data/train.csv', index=False)
val.to_csv('data/val.csv', index=False)

In [21]:
for path in ['data/train.csv', 'data/val.csv']:
    upload_file(path, BUCKET_NAME, path)